# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234838481325                   -0.50    7.0
  2   -7.250245099784       -1.81       -1.41    1.0
  3   -7.251307423185       -2.97       -2.12    3.0
  4   -7.251291907799   +   -4.81       -2.29    4.0
  5   -7.251329124339       -4.43       -2.67    3.0
  6   -7.251336955067       -5.11       -3.02    2.0
  7   -7.251338767889       -5.74       -4.10    1.0
  8   -7.251338792651       -7.61       -4.13    4.0
  9   -7.251338797376       -8.33       -4.44    2.0
 10   -7.251338798092       -9.14       -4.62    2.0
 11   -7.251338798682       -9.23       -5.42    2.0
 12   -7.251338798702      -10.71       -5.82    2.0
 13   -7.251338798704      -11.67       -6.04    2.0
 14   -7.251338798704      -12.10       -6.67    3.0
 15   -7.251338798705      -13.39       -7.13    2.0
 16   -7.251338798705      -14.27       -7.57    3.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.05420332958201644
[ Info: Arnoldi iteration step 2: normres = 0.6746469427889282
[ Info: Arnoldi iteration step 3: normres = 0.7824706816992061
[ Info: Arnoldi iteration step 4: normres = 0.31680061699978784
[ Info: Arnoldi iteration step 5: normres = 0.333763320609745
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.07e-02, 5.72e-02, 2.36e-01, 2.28e-01, 1.03e-02)
[ Info: Arnoldi iteration step 6: normres = 0.403624413733975
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (6.74e-03, 7.82e-02, 3.70e-01, 8.87e-02, 6.29e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07861642823374804
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.76e-04, 1.00e-02, 2.55e-02, 3.01e-02, 5.99e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10851284079074498
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.27e-05